In [1]:
import ROOT
from ROOT import TFile
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
import pandas as pd 
import time
from os import listdir

Welcome to JupyROOT 6.15/01


In [2]:
class particle:
    def __init__(self, pid, fourvector, virtual_photon,PhiPQ):
        #work in frame that is aligned with photon
        #the photon is aligned with z-direction and has 4-momentum q= (0,0,sqrt(Nu2+Q2),Nu) (note q2 = -Q2)        
        Nu = virtual_photon.E()
        Q2 = -virtual_photon.M2()
        proton = ROOT.TLorentzVector()
        proton.SetPxPyPzE(0,0,0, 0.938)
        W = (virtual_photon + proton).M()
        
        photon_pz = np.sqrt(Nu*Nu+Q2) #direction is positive by definition
        self.bcm = photon_pz/(Nu + 0.938)#photon-nucleon center-of-mass velocity 
        self.ycm = 0.5*np.log(( 1+self.bcm)/(1-self.bcm)) #photon-nucleon center-of-mass rapidity
        
        self.LorentzVector = fourvector
        self.E = fourvector.E() #energy in lab frame
        self.vector = fourvector.Vect()
        self.Pt = self.vector.Perp(virtual_photon.Vect().Unit()) #pT with respect to photon direction
        self.Pl  = self.vector.Dot(virtual_photon.Vect().Unit()) #pL with respect to photon direction (in lab frame)
        self.y =  0.5*np.log( (self.E+self.Pl)/(self.E-self.Pl)) #rapidity in lab frame
        self.mT = np.sqrt(fourvector.M2() + self.Pt*self.Pt)
        self.y_star = self.y - self.ycm
        self.Pl_star = self.mT*np.sinh(self.y_star)
        self.Xf = 2.0*self.Pl_star/W 
        self.pid = pid
        self.Zh = self.E/Nu
        self.PhiPQ = PhiPQ*np.pi/180.0
        
        
    def print_properties(self):
        print 'Hello, let me introduce myself, i am particle pid = ' , self.pid, ' with index ', self.index, ', from event  #', self.ievt, ' Nu and W', self.Nu, ' ' , self.W
        print 'zh = ', self.Zh, ' phi_pq= ', self.PhiPQ, ' theta_pq=' , self.ThetaPQ, 'E = ', self.E, ' xf', self.Xf,'Pt ', self.Pt, ' Pl= ', self.Pl, ' rapidity=' ,  self.y
        print 'pid = ' , self.pid

In [15]:

def getDataframes(filename, Target=1,maxevents=1e9):
    dphi = np.array([])
    ParticlesFromPrevious = []
    try:
        myfile = TFile.Open('%s'%filename,'READ')
    except:
        print("could not open file")

        
    myfile.Print()
    mytree = myfile.Get('tree_data')
    e_tree = myfile.Get('e_rec')
    
    
    print filename, ' has ', mytree.GetEntries(), ' entries'
    tupla = {}
    tupla['dphi'] = []
    tupla['dy'] = []
    tupla['z'] = [] 
    tupla['z_asso'] = []
    tupla['xf'] = []
    tupla['xf_asso'] = []
    tupla['y'] = []
    tupla['ycm'] = []

    tupla['y_asso'] = []
    tupla['pid'] = []
    tupla['pid_asso'] = []
    tupla['phi_pq'] = []
    tupla['phi_pq_asso'] = []
    tupla['mass'] = []
    tupla['dipion_pt'] = []
    tupla['missing_mass'] = []
    tupla['t']  = []
    tupla['Q2'] = [] 
    tupla['Nu'] = []
    tupla['W']  = []
    tupla['Xb'] = []
    tupla['u']  = []

    tupla_mix = {}
    tupla_mix['dphi'] = []
    tupla_mix['dy'] = []
    tupla_mix['z'] = []
    tupla_mix['z_asso'] = []
    tupla_mix['xf'] = []
    tupla_mix['xf_asso'] = []
    tupla_mix['y'] = []
    tupla_mix['ycm'] = []

    tupla_mix['y_asso'] = []
    tupla_mix['pid'] = []
    tupla_mix['pid_asso'] = []
    tupla_mix['phi_pq']   = []
    tupla_mix['phi_pq_asso'] = []
    tupla_mix['mass'] = []
    tupla_mix['dipion_pt'] = []
    tupla_mix['missing_mass'] = []
    tupla_mix['t']  = []
    tupla_mix['Q2'] = []
    tupla_mix['Nu'] = []
    tupla_mix['W']  = []
    tupla_mix['Xb'] = []
    tupla_mix['u']  = []
    
    tupla_trigger = {}
    tupla_trigger['pid'] = []
    tupla_trigger['xf'] = []
    tupla_trigger['xf_default'] = []
    tupla_trigger['z']  = []
    tupla_trigger['y']  = []
    tupla_trigger['ycm'] = []
    tupla_trigger['Q2'] = []
    tupla_trigger['Xb'] = []
    tupla_trigger['Nu'] = []
    tupla_trigger['W'] = []
    tupla_trigger['phi_pq'] = []
    tupla_trigger['TargType'] = []
    tupla_trigger['mass'] = []

    start = time.time()

    for ievt  in range(mytree.GetEntries()):
        mytree.GetEntry(ievt)   
        e_tree.GetEntry(ievt)
        if mytree.W<2.05 or mytree.Q2<1.0: continue
        if ievt>maxevents: break
        if mytree.TargType!=Target: continue

        #get electron momentum:
        Pe = np.sqrt(e_tree.Pex*e_tree.Pex + e_tree.Pey*e_tree.Pey+ e_tree.Pez*e_tree.Pez)
        scattered_e = ROOT.TLorentzVector()
        scattered_e.SetPxPyPzE(e_tree.Pex, e_tree.Pey, e_tree.Pez, Pe)
        incoming_e = ROOT.TLorentzVector()
        incoming_e.SetPxPyPzE(0,0,5.014,5.014)
        virtual_photon  = incoming_e - scattered_e 
        virtual_photon_unitvector = virtual_photon.Vect().Unit()
        proton = ROOT.TLorentzVector()
        proton.SetPxPyPzE(0,0,0, 0.938)
        
        #photon-nucleon center-of-mass energy and rapidity
        #the photon is aligned with z-direction and has 4-momentum q= (0,0,sqrt(Nu2+Q2),Nu) (note q2 = -Q2)
        
        W = mytree.W
        Nu = mytree.Nu
        #print 'W ',  W , 'electron W ', e_tree.W
        #print (virtual_photon + proton).M()
        particles = []
        #print ' Entering main loop over particles'
        for i in range(len(mytree.pid)):
            if abs(mytree.pid[i]) !=211: continue
            i_lv = ROOT.TLorentzVector()    
            i_lv.SetPxPyPzE(mytree.Px[i],mytree.Py[i],mytree.Pz[i],mytree.Zh[i]*Nu)
            i_part = particle(mytree.pid[i], i_lv, virtual_photon, mytree.PhiPQ[i] )
            particles.append(i_part)                

            if i_part.Zh > 0.4: #only save triggers and do correlations if they have z>0.4
                tupla_trigger['TargType'].append(mytree.TargType)
                tupla_trigger['pid'].append(i_part.pid)
                tupla_trigger['xf'].append(i_part.Xf)
                tupla_trigger['xf_default'].append(mytree.Xf[i])
                tupla_trigger['z'].append(i_part.Zh)
                tupla_trigger['y'].append(i_part.y_star)
                tupla_trigger['ycm'].append(i_part.ycm)
                tupla_trigger['phi_pq'].append(i_part.PhiPQ)
                tupla_trigger['mass'].append(i_part.LorentzVector.M())
                tupla_trigger['Q2'].append(mytree.Q2)
                tupla_trigger['Xb'].append(mytree.Xb)
                tupla_trigger['Nu'].append(mytree.Nu)
                tupla_trigger['W'].append(mytree.W)

                #print 'mytree.Pt[i] ' , mytree.Pt[i], ' check: ' ,i_part.Vector.Perp(virtual_photon_unitvector)
                for j in range(len(mytree.pid)): 
                    if i==j: continue
                    if abs(mytree.pid[j]) !=211: continue
                    j_lv = ROOT.TLorentzVector()    
                    j_lv.SetPxPyPzE(mytree.Px[j],mytree.Py[j],mytree.Pz[j],mytree.Zh[j]*Nu)
                    j_part = particle(mytree.pid[j], j_lv, virtual_photon, mytree.PhiPQ[j] )
                    
                    dphi = abs(ROOT.TVector2.Phi_mpi_pi(i_part.PhiPQ-j_part.PhiPQ))
                    dy = i_part.y-j_part.y
                    deta = 0
                    dipion = i_part.LorentzVector+j_part.LorentzVector
                    X = (virtual_photon + proton - dipion) #unobserved hadronic system
                    tupla['dphi'].append(dphi)
                    tupla['dy'].append(dy)
                    tupla['z'].append(i_part.Zh)
                    tupla['z_asso'].append(j_part.Zh)
                    tupla['xf'].append(i_part.Xf)
                    tupla['xf_asso'].append(j_part.Xf)
                    tupla['y'].append(i_part.y_star)
                    tupla['ycm'].append(i_part.ycm)
                    tupla['y_asso'].append(j_part.y_star)
                    tupla['pid'].append(i_part.pid)
                    tupla['pid_asso'].append(j_part.pid)
                    tupla['phi_pq'].append(i_part.PhiPQ)
                    tupla['phi_pq_asso'].append(j_part.PhiPQ)
                    tupla['mass'].append(dipion.M())
                    tupla['dipion_pt'].append( dipion.Vect().Perp(virtual_photon_unitvector))
                    tupla['missing_mass'].append(X.M())
                    tupla['t'].append( -(virtual_photon- dipion).M2())
                    tupla['Q2'].append(mytree.Q2)
                    tupla['Xb'].append(mytree.Xb)
                    tupla['Nu'].append(mytree.Nu)
                    tupla['W'].append(mytree.W)
                    tupla['u'].append(-(scattered_e-proton).M2())
                
                    #end loop over secondary loop    
                #print '//////// Entering mixed event correlations with # ', len(ParticlesFromPrevious) , ' paticles in previous event'
                for mixparticle in ParticlesFromPrevious:
                    dipion = i_part.LorentzVector+mixparticle.LorentzVector
                    X = (virtual_photon + proton - dipion)
                    dphi = abs(ROOT.TVector2.Phi_mpi_pi(i_part.PhiPQ-mixparticle.PhiPQ))
                    dy = i_part.y-mixparticle.y
                    deta = 0#i_part.ThetaPQ-mixparticle.ThetaPQ
                    tupla_mix['dphi'].append(dphi)
                    tupla_mix['dy'].append(dy)
                    tupla_mix['z'].append(i_part.Zh)
                    tupla_mix['z_asso'].append(mixparticle.Zh)
                    tupla_mix['xf'].append(i_part.Xf)
                    tupla_mix['xf_asso'].append(mixparticle.Xf)
                    tupla_mix['y'].append(i_part.y)
                    tupla_mix['ycm'].append(i_part.ycm)
                    tupla_mix['y_asso'].append(mixparticle.y)
                    tupla_mix['pid'].append(i_part.pid)
                    tupla_mix['pid_asso'].append(mixparticle.pid)
                    tupla_mix['phi_pq'].append(i_part.PhiPQ)
                    tupla_mix['phi_pq_asso'].append(mixparticle.PhiPQ)
                    tupla_mix['mass'].append(dipion.M())    
                    tupla_mix['dipion_pt'].append( dipion.Vect().Perp(virtual_photon_unitvector))
                    tupla_mix['missing_mass'].append(X.M())
                    tupla_mix['t'].append(-(virtual_photon- dipion).M2())
                    tupla_mix['Q2'].append(mytree.Q2)
                    tupla_mix['Xb'].append(mytree.Xb)
                    tupla_mix['Nu'].append(mytree.Nu)
                    tupla_mix['W'].append(mytree.W)
                    tupla_mix['u'].append( -(scattered_e-proton).M2())
                    
                    
        ParticlesFromPrevious = particles
        #print ' going for next event'    
        #print ' particles in event', len(particles
        ##end loop over events correlations    
    end = time.time()
    print 'Processed in',  end-start, 'seconds'
    
    df = pd.DataFrame(tupla)
    df_mix= pd.DataFrame(tupla_mix)
    df_trigger = pd.DataFrame(tupla_trigger)
    print 'Number of triggers with z>0.4,  ', df.query('z>0.4').shape[0]
    print 'Number of pairs with z>0.4, ' ,df_trigger.query('z>0.4').shape[0] 
    myfile.Close()
    return df, df_mix, df_trigger

In [4]:
#for a particle on-shell:
# E = mT cosh y 
# pz = mT sinh y
#mT2 = m2 + pt2
# rapidity in the ion-at-rest frame, with the photon direction is 
# y = 0.5 * ln( (E+pz)/(E-pz))
#rapidity of the target in the lab frame = 0 (at rest)
# y = y* + ycm
#bcm = az + bz/ a0 + b0 
#bcm = photon_z / Nu + m_{p}
#ycm = 0.5 * ln( 1+bcm/1-bcm)
#energy of the photon = Nu
#momentum of the photon (magnitude) = sqrt(Nu2 + Q2) 
#q2 = -Q2. (four-momentum transfer)

## Create dataframe df

In [5]:
df = {}

## Pb data

In [9]:
path = '/mnt/c/Users/marratia/Linux/Pb_data/'

Files = listdir(path) 
df['Pb'],df['Pb_mix'], df['Pb_trigger'] = [None,None,None]
df['D_Pb'],df['D_Pb_mix'], df['D_Pb_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    pairs, pairs_mix, trigger = getDataframes(filename,Target=2)
    df['Pb'] = pd.concat([ df['Pb'], pairs])
    df['Pb_mix'] = pd.concat([ df['Pb_mix'], pairs_mix])
    df['Pb_trigger'] = pd.concat([ df['Pb_trigger'], trigger])
    pairs, pairs_mix, trigger = getDataframes(filename,Target=1)
    df['D_Pb'] = pd.concat([ df['D_Pb'], pairs])
    df['D_Pb_mix'] = pd.concat([ df['D_Pb_mix'], pairs_mix])
    df['D_Pb_trigger'] = pd.concat([ df['D_Pb_trigger'], trigger])

/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41762.root  has  200510  entries


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log
  del sys.path[0]


Processed in 23.4708499908 seconds
Number of triggers with z>0.4,   919
Number of pairs with z>0.4,  4267
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41762.root  has  200510  entries
Processed in 54.7801690102 seconds
Number of triggers with z>0.4,   4602
Number of pairs with z>0.4,  18764
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41763.root  has  190818  entries
Processed in 21.3627510071 seconds
Number of triggers with z>0.4,   848
Number of pairs with z>0.4,  4064
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41763.root  has  190818  entries
Processed in 52.3791618347 seconds
Number of triggers with z>0.4,   4366
Number of pairs with z>0.4,  17829
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41765.root  has  23948  entries
Processed in 2.70751094818 seconds
Number of triggers with z>0.4,   100
Number of pairs with z>0.4,  525
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41765.root  has  23948  entries
Processed in 6.57368302345 seconds
Number of

Processed in 55.3015789986 seconds
Number of triggers with z>0.4,   4621
Number of pairs with z>0.4,  18914
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41809.root  has  194775  entries
Processed in 21.6772961617 seconds
Number of triggers with z>0.4,   872
Number of pairs with z>0.4,  4108
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41809.root  has  194775  entries
Processed in 53.8717989922 seconds
Number of triggers with z>0.4,   4522
Number of pairs with z>0.4,  18322
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41810.root  has  198405  entries
Processed in 22.1789281368 seconds
Number of triggers with z>0.4,   885
Number of pairs with z>0.4,  4164
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41810.root  has  198405  entries
Processed in 54.0923790932 seconds
Number of triggers with z>0.4,   4649
Number of pairs with z>0.4,  18614
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41811.root  has  200656  entries
Processed in 22.45510602 seconds
Number

Processed in 22.5712871552 seconds
Number of triggers with z>0.4,   957
Number of pairs with z>0.4,  4350
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41830.root  has  209045  entries
Processed in 55.9133298397 seconds
Number of triggers with z>0.4,   4867
Number of pairs with z>0.4,  19775
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41831.root  has  213141  entries
Processed in 23.4640209675 seconds
Number of triggers with z>0.4,   970
Number of pairs with z>0.4,  4541
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41831.root  has  213141  entries
Processed in 56.1984198093 seconds
Number of triggers with z>0.4,   4913
Number of pairs with z>0.4,  19912
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41832.root  has  199844  entries
Processed in 21.5448420048 seconds
Number of triggers with z>0.4,   880
Number of pairs with z>0.4,  4126
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41832.root  has  199844  entries
Processed in 53.7434251308 seconds
Number

Processed in 51.4881420135 seconds
Number of triggers with z>0.4,   4392
Number of pairs with z>0.4,  18097
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41867.root  has  191129  entries
Processed in 20.6568441391 seconds
Number of triggers with z>0.4,   820
Number of pairs with z>0.4,  4050
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41867.root  has  191129  entries
Processed in 50.7356188297 seconds
Number of triggers with z>0.4,   4356
Number of pairs with z>0.4,  17875
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41868.root  has  192813  entries
Processed in 20.6240069866 seconds
Number of triggers with z>0.4,   798
Number of pairs with z>0.4,  4035
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41868.root  has  192813  entries
Processed in 51.9300539494 seconds
Number of triggers with z>0.4,   4415
Number of pairs with z>0.4,  18350
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41869.root  has  194121  entries
Processed in 21.0315060616 seconds
Numb

Processed in 14.7740330696 seconds
Number of triggers with z>0.4,   602
Number of pairs with z>0.4,  2871
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41889.root  has  136963  entries
Processed in 36.4926431179 seconds
Number of triggers with z>0.4,   3151
Number of pairs with z>0.4,  12662
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41890.root  has  121925  entries
Processed in 13.2755389214 seconds
Number of triggers with z>0.4,   529
Number of pairs with z>0.4,  2522
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41890.root  has  121925  entries
Processed in 32.4625298977 seconds
Number of triggers with z>0.4,   2749
Number of pairs with z>0.4,  11458
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41891.root  has  134080  entries
Processed in 14.9214150906 seconds
Number of triggers with z>0.4,   576
Number of pairs with z>0.4,  2892
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41891.root  has  134080  entries
Processed in 35.7487590313 seconds
Number

Processed in 38.1254060268 seconds
Number of triggers with z>0.4,   3182
Number of pairs with z>0.4,  13422
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41916.root  has  165710  entries
Processed in 17.7894399166 seconds
Number of triggers with z>0.4,   710
Number of pairs with z>0.4,  3533
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41916.root  has  165710  entries
Processed in 44.475826025 seconds
Number of triggers with z>0.4,   3839
Number of pairs with z>0.4,  15560
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41917.root  has  79618  entries
Processed in 8.50430488586 seconds
Number of triggers with z>0.4,   367
Number of pairs with z>0.4,  1611
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41917.root  has  79618  entries
Processed in 21.1782240868 seconds
Number of triggers with z>0.4,   1795
Number of pairs with z>0.4,  7415
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41918.root  has  59842  entries
Processed in 6.51000595093 seconds
Number of

Processed in 16.5965161324 seconds
Number of triggers with z>0.4,   608
Number of pairs with z>0.4,  3288
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41953.root  has  152694  entries
Processed in 40.3781900406 seconds
Number of triggers with z>0.4,   3465
Number of pairs with z>0.4,  14201
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41954.root  has  195498  entries
Processed in 21.3577740192 seconds
Number of triggers with z>0.4,   903
Number of pairs with z>0.4,  4198
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41954.root  has  195498  entries
Processed in 51.9991178513 seconds
Number of triggers with z>0.4,   4485
Number of pairs with z>0.4,  18456
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41955.root  has  149865  entries
Processed in 16.5252709389 seconds
Number of triggers with z>0.4,   661
Number of pairs with z>0.4,  3122
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41955.root  has  149865  entries
Processed in 40.1538059711 seconds
Number

Processed in 24.5826539993 seconds
Number of triggers with z>0.4,   1134
Number of pairs with z>0.4,  4579
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41976.root  has  49058  entries
Processed in 9.9154920578 seconds
Number of triggers with z>0.4,   242
Number of pairs with z>0.4,  1048
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41976.root  has  49058  entries
Processed in 24.7268428802 seconds
Number of triggers with z>0.4,   1146
Number of pairs with z>0.4,  4665
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41977.root  has  161751  entries
Processed in 33.0855271816 seconds
Number of triggers with z>0.4,   697
Number of pairs with z>0.4,  3371
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41977.root  has  161751  entries
Processed in 82.393255949 seconds
Number of triggers with z>0.4,   3833
Number of pairs with z>0.4,  15342
/mnt/c/Users/marratia/Linux/Pb_data/CFFTree_Pb_data_41981.root  has  178012  entries
Processed in 35.9385249615 seconds
Number of 

## Fe data

In [6]:
path = '/mnt/c/Users/marratia/Linux/Fe_data/'
Files = listdir(path) 
df['Fe'],df['Fe_mix'], df['Fe_trigger'] = [None,None,None]
df['D_Fe'],df['D_Fe_mix'], df['D_Fe_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    pairs, pairs_mix, trigger = getDataframes(filename,Target=2)
    df['Fe'] = pd.concat([ df['Fe'], pairs])
    df['Fe_mix'] = pd.concat([ df['Fe_mix'], pairs_mix])
    df['Fe_trigger'] = pd.concat([ df['Fe_trigger'], trigger])
    pairs, pairs_mix, trigger = getDataframes(filename,Target=1)
    df['D_Fe'] = pd.concat([ df['D_Fe'], pairs])
    df['D_Fe_mix'] = pd.concat([ df['D_Fe_mix'], pairs_mix])
    df['D_Fe_trigger'] = pd.concat([ df['D_Fe_trigger'], trigger])

/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41146.root  has  220529  entries


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log
  del sys.path[0]


Processed in 91.9589111805 seconds
Number of triggers with z>0.4,   2148
Number of pairs with z>0.4,  9525
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41146.root  has  220529  entries
Processed in 99.9998281002 seconds
Number of triggers with z>0.4,   3833
Number of pairs with z>0.4,  15127
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41147.root  has  208300  entries
Processed in 80.2083778381 seconds
Number of triggers with z>0.4,   2122
Number of pairs with z>0.4,  9032
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41147.root  has  208300  entries
Processed in 83.5817039013 seconds
Number of triggers with z>0.4,   3634
Number of pairs with z>0.4,  14653
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41148.root  has  182293  entries
Processed in 62.0741379261 seconds
Number of triggers with z>0.4,   1905
Number of pairs with z>0.4,  7779
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41148.root  has  182293  entries
Processed in 82.7893259525 seconds
Num

Processed in 94.3892321587 seconds
Number of triggers with z>0.4,   3847
Number of pairs with z>0.4,  14865
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41176.root  has  187146  entries
Processed in 65.5214891434 seconds
Number of triggers with z>0.4,   1849
Number of pairs with z>0.4,  7790
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41176.root  has  187146  entries
Processed in 80.0781879425 seconds
Number of triggers with z>0.4,   3380
Number of pairs with z>0.4,  12804
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41177.root  has  214795  entries
Processed in 108.631290913 seconds
Number of triggers with z>0.4,   2127
Number of pairs with z>0.4,  8903
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41177.root  has  214795  entries
Processed in 168.997875929 seconds
Number of triggers with z>0.4,   3820
Number of pairs with z>0.4,  14737
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41178.root  has  211186  entries
Processed in 112.331161976 seconds
Nu

Processed in 9.42105507851 seconds
Number of triggers with z>0.4,   545
Number of pairs with z>0.4,  2312
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41212.root  has  54469  entries
Processed in 10.9531629086 seconds
Number of triggers with z>0.4,   938
Number of pairs with z>0.4,  3720
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41213.root  has  120173  entries
Processed in 20.81980896 seconds
Number of triggers with z>0.4,   1189
Number of pairs with z>0.4,  5123
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41213.root  has  120173  entries
Processed in 23.8711030483 seconds
Number of triggers with z>0.4,   2047
Number of pairs with z>0.4,  8168
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41214.root  has  27796  entries
Processed in 4.78667998314 seconds
Number of triggers with z>0.4,   266
Number of pairs with z>0.4,  1180
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41214.root  has  27796  entries
Processed in 5.56883502007 seconds
Number of tri

...

Processed in 13.0175251961 seconds
Number of triggers with z>0.4,   689
Number of pairs with z>0.4,  3196
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41409.root  has  73943  entries
Processed in 15.2724339962 seconds
Number of triggers with z>0.4,   1288
Number of pairs with z>0.4,  5111
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41410.root  has  102006  entries
Processed in 17.7023229599 seconds
Number of triggers with z>0.4,   1011
Number of pairs with z>0.4,  4452
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41410.root  has  102006  entries
Processed in 20.6872580051 seconds
Number of triggers with z>0.4,   1801
Number of pairs with z>0.4,  7148
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41413.root  has  185944  entries
Processed in 32.0734140873 seconds
Number of triggers with z>0.4,   1790
Number of pairs with z>0.4,  7906
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41413.root  has  185944  entries
Processed in 37.6685299873 seconds
Number 

Processed in 38.096706152 seconds
Number of triggers with z>0.4,   3198
Number of pairs with z>0.4,  13054
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41450.root  has  188712  entries
Processed in 32.6670389175 seconds
Number of triggers with z>0.4,   1746
Number of pairs with z>0.4,  7952
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41450.root  has  188712  entries
Processed in 38.2325470448 seconds
Number of triggers with z>0.4,   3231
Number of pairs with z>0.4,  12873
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41451.root  has  192966  entries
Processed in 34.1414241791 seconds
Number of triggers with z>0.4,   1848
Number of pairs with z>0.4,  8357
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41451.root  has  192966  entries
Processed in 40.6817610264 seconds
Number of triggers with z>0.4,   3398
Number of pairs with z>0.4,  13428
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41452.root  has  201862  entries
Processed in 34.9617660046 seconds
Num

Processed in 3.49778199196 seconds
Number of triggers with z>0.4,   208
Number of pairs with z>0.4,  855
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41475.root  has  20089  entries
Processed in 4.0522749424 seconds
Number of triggers with z>0.4,   319
Number of pairs with z>0.4,  1344
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41476.root  has  39418  entries
Processed in 6.89270091057 seconds
Number of triggers with z>0.4,   388
Number of pairs with z>0.4,  1741
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41476.root  has  39418  entries
Processed in 7.82351493835 seconds
Number of triggers with z>0.4,   628
Number of pairs with z>0.4,  2778
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41478.root  has  186675  entries
Processed in 32.9460759163 seconds
Number of triggers with z>0.4,   1770
Number of pairs with z>0.4,  8042
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41478.root  has  186675  entries
Processed in 37.699406147 seconds
Number of trigg

Processed in 37.6148798466 seconds
Number of triggers with z>0.4,   3219
Number of pairs with z>0.4,  13074
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41514.root  has  186995  entries
Processed in 32.5872929096 seconds
Number of triggers with z>0.4,   1723
Number of pairs with z>0.4,  7877
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41514.root  has  186995  entries
Processed in 37.39128685 seconds
Number of triggers with z>0.4,   3286
Number of pairs with z>0.4,  12890
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41515.root  has  185985  entries
Processed in 32.6164200306 seconds
Number of triggers with z>0.4,   1751
Number of pairs with z>0.4,  7921
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41515.root  has  185985  entries
Processed in 37.5772438049 seconds
Number of triggers with z>0.4,   3191
Number of pairs with z>0.4,  12796
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41516.root  has  183332  entries
Processed in 31.7807178497 seconds
Numb

Processed in 33.0232999325 seconds
Number of triggers with z>0.4,   1743
Number of pairs with z>0.4,  8062
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41539.root  has  186701  entries
Processed in 37.3643369675 seconds
Number of triggers with z>0.4,   3195
Number of pairs with z>0.4,  12974
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41540.root  has  185868  entries
Processed in 32.6765999794 seconds
Number of triggers with z>0.4,   1780
Number of pairs with z>0.4,  8029
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41540.root  has  185868  entries
Processed in 37.2180747986 seconds
Number of triggers with z>0.4,   3130
Number of pairs with z>0.4,  12854
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41541.root  has  185608  entries
Processed in 32.5754029751 seconds
Number of triggers with z>0.4,   1820
Number of pairs with z>0.4,  7982
/mnt/c/Users/marratia/Linux/Fe_data/CFFTree_Fe_data_41541.root  has  185608  entries
Processed in 36.9472990036 seconds
Num

# C data

In [12]:
path = '/mnt/c/Users/marratia/Linux/C_data/'
Files = listdir(path) 
df['C'],df['C_mix'], df['C_trigger'] = [None,None,None]
df['D_C'],df['D_C_mix'], df['D_C_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    pairs, pairs_mix, trigger = getDataframes(filename,Target=2)
    df['C'] = pd.concat([ df['C'], pairs])
    df['C_mix'] = pd.concat([ df['C_mix'], pairs_mix])
    df['C_trigger'] = pd.concat([ df['C_trigger'], trigger])
    pairs, pairs_mix, trigger = getDataframes(filename,Target=1)
    df['D_C'] = pd.concat([ df['D_C'], pairs])
    df['D_C_mix'] = pd.concat([ df['D_C_mix'], pairs_mix])
    df['D_C_trigger'] = pd.concat([ df['D_C_trigger'], trigger])

/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42011.root  has  195907  entries


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log
  del sys.path[0]


Processed in 35.7671558857 seconds
Number of triggers with z>0.4,   2334
Number of pairs with z>0.4,  9942
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42011.root  has  195907  entries
Processed in 38.963173151 seconds
Number of triggers with z>0.4,   3106
Number of pairs with z>0.4,  12810
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42012.root  has  110697  entries
Processed in 20.312513113 seconds
Number of triggers with z>0.4,   1320
Number of pairs with z>0.4,  5580
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42012.root  has  110697  entries
Processed in 22.3157010078 seconds
Number of triggers with z>0.4,   1786
Number of pairs with z>0.4,  7297
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42013.root  has  183077  entries
Processed in 33.6725380421 seconds
Number of triggers with z>0.4,   2290
Number of pairs with z>0.4,  9443
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42013.root  has  183077  entries
Processed in 36.1527810097 seconds
Number of trigge

/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42041.root  has  216793  entries
Processed in 41.1457750797 seconds
Number of triggers with z>0.4,   2771
Number of pairs with z>0.4,  11090
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42041.root  has  216793  entries
Processed in 43.9691951275 seconds
Number of triggers with z>0.4,   3494
Number of pairs with z>0.4,  14213
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42042.root  has  220776  entries
Processed in 39.7604570389 seconds
Number of triggers with z>0.4,   2697
Number of pairs with z>0.4,  11024
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42042.root  has  220776  entries
Processed in 44.0518510342 seconds
Number of triggers with z>0.4,   3608
Number of pairs with z>0.4,  14715
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42043.root  has  225470  entries
Processed in 41.0438301563 seconds
Number of triggers with z>0.4,   2711
Number of pairs with z>0.4,  11318
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_

/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42064.root  has  216647  entries
Processed in 43.5709738731 seconds
Number of triggers with z>0.4,   3681
Number of pairs with z>0.4,  14596
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42065.root  has  216124  entries
Processed in 39.2829339504 seconds
Number of triggers with z>0.4,   2598
Number of pairs with z>0.4,  11001
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42065.root  has  216124  entries
Processed in 43.318625927 seconds
Number of triggers with z>0.4,   3537
Number of pairs with z>0.4,  14204
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42066.root  has  223736  entries
Processed in 40.7443089485 seconds
Number of triggers with z>0.4,   2776
Number of pairs with z>0.4,  11348
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42066.root  has  223736  entries
Processed in 44.6750948429 seconds
Number of triggers with z>0.4,   3581
Number of pairs with z>0.4,  14652
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_d

/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42086.root  has  220890  entries
Processed in 73.3269159794 seconds
Number of triggers with z>0.4,   2622
Number of pairs with z>0.4,  11202
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42086.root  has  220890  entries
Processed in 79.2325458527 seconds
Number of triggers with z>0.4,   3584
Number of pairs with z>0.4,  14711
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42087.root  has  223527  entries
Processed in 73.4461619854 seconds
Number of triggers with z>0.4,   2860
Number of pairs with z>0.4,  11452
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42087.root  has  223527  entries
Processed in 80.5660769939 seconds
Number of triggers with z>0.4,   3645
Number of pairs with z>0.4,  14925
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42088.root  has  222781  entries
Processed in 73.3406710625 seconds
Number of triggers with z>0.4,   2844
Number of pairs with z>0.4,  11413
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_

/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42114.root  has  226762  entries
Processed in 83.0291938782 seconds
Number of triggers with z>0.4,   3918
Number of pairs with z>0.4,  15157
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42115.root  has  227641  entries
Processed in 75.2514679432 seconds
Number of triggers with z>0.4,   2785
Number of pairs with z>0.4,  11352
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42115.root  has  227641  entries
Processed in 82.8485629559 seconds
Number of triggers with z>0.4,   3783
Number of pairs with z>0.4,  15007
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42116.root  has  219852  entries
Processed in 72.4376451969 seconds
Number of triggers with z>0.4,   2610
Number of pairs with z>0.4,  10876
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42116.root  has  219852  entries
Processed in 80.6005370617 seconds
Number of triggers with z>0.4,   3744
Number of pairs with z>0.4,  14641
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_

/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42136.root  has  210191  entries
Processed in 70.731336832 seconds
Number of triggers with z>0.4,   2593
Number of pairs with z>0.4,  10607
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42136.root  has  210191  entries
Processed in 79.4363720417 seconds
Number of triggers with z>0.4,   3549
Number of pairs with z>0.4,  13986
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42137.root  has  217858  entries
Processed in 72.5417051315 seconds
Number of triggers with z>0.4,   2600
Number of pairs with z>0.4,  10779
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42137.root  has  217858  entries
Processed in 80.7345490456 seconds
Number of triggers with z>0.4,   3487
Number of pairs with z>0.4,  14294
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_data_42138.root  has  208314  entries
Processed in 70.5543048382 seconds
Number of triggers with z>0.4,   2549
Number of pairs with z>0.4,  10560
/mnt/c/Users/marratia/Linux/C_data/CFFTree_C_d

In [13]:
df['Pb'].head()

,Nu,Q2,W,Xb,dipion_pt,dphi,dy,mass,missing_mass,phi_pq,...,pid_asso,t,u,xf,xf_asso,y,y_asso,ycm,z,z_asso
0,3.663211,1.614811,2.477846,0.234910,0.403655,0.309819,0.237864,0.292044,1.474685,-1.867758,...,211,0.829680,1.654236,0.319164,0.203544,1.188384,0.950520,1.230356,0.410388,0.280462
1,2.755150,1.344208,2.169402,0.259993,0.281854,2.548473,0.232499,0.875773,0.876162,2.342740,...,-211,0.333496,3.357759,0.286801,0.242360,0.720687,0.488187,1.125225,0.466954,0.490228
2,2.755150,1.344208,2.169402,0.259993,0.281854,2.548473,-0.232499,0.875773,0.876162,-1.391973,...,211,0.333496,3.357759,0.242360,0.286801,0.488187,0.720687,1.125225,0.490228,0.466954
3,2.929695,1.513593,2.205553,0.275314,0.328766,2.212400,0.725551,0.769407,1.078895,2.714442,...,211,0.669144,3.030312,0.380175,0.069979,0.922699,0.197148,1.161588,0.551269,0.275278
4,4.309378,1.797300,2.677647,0.222253,0.164371,2.669166,1.264960,0.896089,1.493518,-2.866092,...,-211,1.005755,0.442027,0.436211,-0.014247,1.214069,-0.050891,1.293602,0.545376,0.163135


In [ ]:
plt.hist2d(df['Pb_trigger'].query('z<0.3 and abs(xf)<1.0')['y_star'],df['Pb_trigger'].query('z<0.3 and abs(xf)<1.0')['xf'],bins=[100,100])

In [ ]:
plt.hist(df['Pb_trigger'].query('z<0.3')['y_star'],range=(-4,4),bins=100,alpha=0.4,normed=True)
plt.hist(df['Pb_trigger'].query('z>0.4')['y_star'],range=(-4,4),bins=100,alpha=0.4,normed=True)
plt.hist(df['Pb_trigger']['y_star'],range=(-4,4),bins=100,alpha=0.4,normed=True)

plt.show()

In [ ]:
plt.hist(df['Pb'].query('z>0.5')['dy'],range=(-4,4),bins=50,alpha=0.4,normed=True)
print df['Pb'].query('z>0.5').shape[0]

In [7]:
import root_pandas
from root_pandas import read_root
from root_pandas import to_root 

## Save dataframes to ROOT files

In [11]:
to_root(df['Pb'],'Pairs_Pb.root', key='Pb')
to_root(df['Pb_mix'],'Pairs_Pb.root', key='Pb_mix',mode='a')
to_root(df['Pb_trigger'],'Pairs_Pb.root', key='Pb_trigger', mode='a')
to_root(df['D_Pb'],'Pairs_Pb.root', key='D_Pb',mode='a')
to_root(df['D_Pb_mix'],'Pairs_Pb.root', key='D_Pb_mix',mode='a')
to_root(df['D_Pb_trigger'],'Pairs_Pb.root', key='D_Pb_trigger', mode='a')

In [8]:
to_root(df['Fe'],'Pairs_Fe.root', key='Fe')
to_root(df['Fe_mix'],'Pairs_Fe.root', key='Fe_mix',mode='a')
to_root(df['Fe_trigger'],'Pairs_Fe.root', key='Fe_trigger', mode='a')
to_root(df['D_Fe'],'Pairs_Fe.root', key='D_Fe',mode='a')
to_root(df['D_Fe_mix'],'Pairs_Fe.root', key='D_Fe_mix',mode='a')
to_root(df['D_Fe_trigger'],'Pairs_Fe.root', key='D_Fe_trigger', mode='a')

In [14]:
to_root(df['C'],'Pairs_C.root', key='C')
to_root(df['C_mix'],'Pairs_C.root', key='C_mix',mode='a')
to_root(df['C_trigger'],'Pairs_C.root', key='C_trigger', mode='a')
to_root(df['D_C'],'Pairs_C.root', key='D_C',mode='a')
to_root(df['D_C_mix'],'Pairs_C.root', key='D_C_mix',mode='a')
to_root(df['D_C_trigger'],'Pairs_C.root', key='D_C_trigger', mode='a')